In [ ]:
import torch
import torchvision
# CIFAR-10 데이터셋 다운로드 및 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
# 다운이 코드 참고해서 block화 읽기
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet(nn.Module):
    
    def __init__(self):
        
        super(ResNet, self).__init__()
        
        # __init__ 메소드 안에 함수를 정의하기도 하는군
        def block(in_channels_, out_channels, kernel_size = 3, padding = 1):
            
            # downsampling 할 때는 stride가 2
            if in_channels == out_channels:
                stride = 1
            
            else:
                stride = 2
                
            layer = nn.Sequential(
                
                nn.Conv2d(in_channels = in_channels_, out_channels = out_channels, kernel_size = kernel_size,
                          stride = stride, padding = padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
                
                nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = kernel_size,
                          stride = stride, padding = padding)
                nn.BatchNorm2d(out_channels)
                
            )
            
            return layer
        
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size = 7, )
                
            
        
        

In [ ]:
# 다운이 코드
#%%writefile "../model/ResNet_Residual.py"

# import modules
import os
import torch
from torch import nn
from torchvision import transforms

class ResNet_Residual(nn.Module):

    def __init__(self):
        # super함수는 class의 부모 class인 nn.Module을 초기화
        super(ResNet_Residual, self).__init__()

        # Plain이나 Resnet이나 블록 형태는 같다. 
        def block(in_channels, out_channels, kernel_size = 3, padding = 1):
            # downsampling할 때 stride 가 2
            if in_channels == out_channels:
                stride = 1
            
            else:
                stride = 2

            # layer block -> ResNet에서는 이부분이 Residual block이 될 예정
            layer = nn.Sequential(
                nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride = stride, padding = padding),
                nn.BatchNorm2d(out_channels),
                nn.Relu(),

                nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = kernel_size, padding = padding),
                nn.BatchNorm2d(out_channels)
            )

            return layer

        # 맨 처음 7x7과 풀링(VGG19에서 Maxpooling 사용)
        self.seven_conv = nn.Conv2d(in_channels = 3, out_channels = 64, padding = 3)
        self.pool = nn.MaxPool2d(kernel_size = 2)
        
        # 64층
        self.block64_1 = block(in_channels = 64, out_channels = 64)
        self.block64_2 = block(in_channels = 64, out_channels = 64)
        self.block64_3 = block(in_channels = 64, out_channels = 64)
        
        # 128층
        self.block128_1 = block(in_channels = 64, out_channels = 128)
        self.block128_2 = block(in_channels = 128, out_channels = 128)
        self.block128_3 = block(in_channels = 128, out_channels = 128)
        self.block128_4 = block(in_channels = 128, out_channels = 128)
        
        # 256층
        self.block256_1 = block(in_channels = 128, out_channels = 256)
        self.block256_2 = block(in_channels = 256, out_channels = 256)
        self.block256_3 = block(in_channels = 256, out_channels = 256)
        self.block256_4 = block(in_channels = 256, out_channels = 256)
        self.block256_5 = block(in_channels = 256, out_channels = 256)
        self.block256_6 = block(in_channels = 256, out_channels = 256)
        
        # 512층
        self.block512_1 = block(in_channels = 256, out_channels = 512)
        self.block512_2 = block(in_channels = 512, out_channels = 512)
        self.block512_3 = block(in_channels = 512, out_channels = 512)

        # avgpool
        self.average_pool = nn.AdaptiveAvgPool2d(kernel_size = 1)

        # fully connected layer
        self.fully_connected_layer = nn.Linear(in_channels = 512, out_channels = 1000)

        # 추가 relu
        self.relu = nn.Relu()

        # 보정
        self.prejection1 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 1)
        self.prejection2 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 1)
        self.prejection3 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 1)

    def forward(self, x):
        
        # plain -> 단순 연결만 진행하면 됨
        x = self.seven_conv(x)
        x = self.pool(x)

        # 64
        res1 = self.block64_1(x)
        x = res1 + x
        x = self.relu(x)

        res2 = self.block64_2(x)
        x = res2 + x
        x = self.relu(x)

        res3 = self.block64_3(x)
        x = res3 + x
        x = self.relu(x)

        # 128
        res1 = self.block128_1(x)
        x = self.prejection1(x)
        x = res1 + x
        x = self.relu(x)

        res2 = self.block128_2(x)
        x = res2 + x
        x = self.relu(x)

        res3 = self.block128_3(x)
        x = res3 + x
        x = self.relu(x)

        res4 = self.block128_4(x)
        x = res4 + x
        x = self.relu(x)

        # 256
        res1 = self.block256_1(x)
        x = self.prejection2(x)
        x = res1 + x
        x = self.relu(x)

        res2 = self.block256_2(x)
        x = res2 + x
        x = self.relu(x)

        res3 = self.block256_3(x)
        x = res3 + x
        x = self.relu(x)

        res4 = self.block256_4(x)
        x = res4 + x
        x = self.relu(x)

        res5 = self.block256_5(x)
        x = res5 + x
        x = self.relu(x)

        res6 = self.block256_6(x)
        x = res6 + x
        x = self.relu(x)

        # 512
        res1 = self.block512_1(x)
        x = self.prejection3(x)
        x = res1 + x
        x = self.relu(x)

        res2 = self.block512_2(x)
        x = res2 + x
        x = self.relu(x)

        res3 = self.block512_3(x)
        x = res3 + x
        x = self.relu(x)

        # 마지막 결과물
        x = self.average_pool(x)
        x = self.fully_connected_layer(x)

        return x

In [ ]:
# 상혁 코드
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels), # 학습 빠르게 하기 위함
            nn.ReLU(),
            
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels) # 학습 빠르게 하기 위함
        )
        
        if self.stride != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.out_channels)
            )
            
    # Block 하나 거칠 때마다 이미지 사이즈가 줄고, 채널 수 늘어나는 구조
    # 처음 들어오는 X와 Block을 거친 출력값 out의 크기가 같아야함.
    # X, out의 크기가 차이가 날 경우 크기를 동일하게 하기 위해 별도 conv 연산을 진행하여
    # 입력 크기와 출력 크기를 같게 맞춤 
    def forward(self, x):
        out = self.conv_block(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            x = self.downsample(x)
        out = F.relu(x + out)
        return out
    
    

class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        # cifar 10의 경우 32 x 32 
        self.layer1 = self.make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(512, num_blocks[3], stride=2)

        self.gap = nn.AvgPool2d(4)
        self.fc = nn.Linear(512, num_classes)


    def make_layer(self, out_channels, num_block, stride):
        strides = [stride] + [1] * (num_block - 1)
        layers = []
        
        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels
        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)

        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
def modeltype(model):
    if model == 'resnet18':
        return ResNet([2, 2, 2, 2])
    elif model == 'resnet34':
        return ResNet([3, 4, 6, 3])

In [ ]:
# 준혁 코드
import torch.nn as nn
import torch.nn.functional as F

class res_net():
    def __init__(self, num_classes=10):
        super(res_net, self).__init__()
        
        def CBR2d_Blocks(first_ch, inout_ch, kernel_size=3, stride=1, padding=1, bias=True):
            return nn.Sequential(nn.Conv2d(in_channels=first_ch, out_channels=inout_ch, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
                      nn.BatchNorm2d(inout_ch),
                      nn.ReLU(),
                      nn.Conv2d(in_channels=inout_ch, out_channels=inout_ch, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
                      nn.BatchNorm2d(inout_ch),
                      nn.ReLU())

        self.first_layer = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=0, bias=True),
                                         nn.ReLU(),
                                         nn.Maxpool2d(kernel_size=2),
                                         nn.ReLU()
                                         )
        

        self.second_layer = CBR2d_Blocks(first_ch=64, inout_ch=64)
        self.to_third_layer = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=2, padding=0, bias=True)

        self.third_layer_1 = CBR2d_Blocks(first_ch=64, inout_ch=128)
        self.third_layer_2 = CBR2d_Blocks(first_ch=128, inout_ch=128)
        self.to_fourth_layer = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=2, padding=0, bias=True)
      

        self.forth_layer_1 = CBR2d_Blocks(fisrt_ch=128, inout_ch=256)
        self.forth_layer_2 = CBR2d_Blocks(first_ch=256, inout_ch=256)
        self.to_fifth_layer = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=2, padding=0, bias=True)

        self.fifth_layer_1 = CBR2d_Blocks(first_ch=256, inout_ch=512)
        self.fifth_layer_2 = CBR2d_Blocks(first_ch=512, inout_ch=512)


        self.last_layer = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):

        # first_layer
        x = self.first_layer(x)

        # 64 channels
        identity = x
        x = self.second_layer(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.second_layer(x)
        x += identity
        x = nn.ReLU(x)
        identity = x
        
        x = self.second_layer(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        # identity to next layer
        identity = self.to_third_layer(identity)

        # 128 channels
        x = self.third_layer_1(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.third_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.third_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.third_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        # identity to next layer
        identity = self.to_fifth_layer(identity)

        # 256 channels
        x = self.fourth_layer_1(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fourth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fourth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fourth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fourth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fourth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        # identity to next layer
        identity = self.to_fifth_layer(identity)

        # 512 channels
        x = self.fifth_layer_1(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fifth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        x = self.fifth_layer_2(x)
        x += identity
        x = nn.ReLU(x)
        identity = x

        # last_layer
        x = self.last_layer(x)
        return x
    

In [3]:
%%writefile "ResNet.py"

import torch
import torch.nn as nn
import torch.nn.functional as F


class ResNet(nn.Module):
    
    def __init__(self):
        super(ResNet, self).__init_()
        
        # filter size 7
        self.conv1 = nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3) # in_channels self.parameter로 추가하는 게 낫지 않을까
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        # 64 channels
        self.conv2_1 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1) # stride default = 1
        self.bn2_1 = nn.BatchNorm2d(64)
        self.relu2_1 = nn.ReLU()
        
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.bn2_2 = nn.BatchNorm2d(64)
        self.relu2_2 = nn.ReLU()
        
        self.conv2_3 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.bn2_3 = nn.BatchNorm2d(64)
        self.relu2_3 = nn.ReLU()
        
        self.conv2_4 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.bn2_4 = nn.BatchNorm2d(64)
        self.relu2_4 = nn.ReLU()
        
        self.conv2_5 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.bn2_5 = nn.BatchNorm2d(64)
        self.relu2_5 = nn.ReLU()
        
        self.conv2_6 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.bn2_6 = nn.BatchNorm2d(64)
        self.relu2_6 = nn.ReLU()
        
        # 128 channels
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size = 3, stride = 2)
        self.bn3_1 = nn.BatchNorm2d(128)
        self.relu3_1 = nn.ReLU()
        
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_2 = nn.BatchNorm2d(128)
        self.relu3_2 = nn.ReLU()
        
        self.conv3_proj = nn.Conv2d(64, 128, kernel_size = 1) # for projection shortcut
        
        self.conv3_3 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_3 = nn.BatchNorm2d(128)
        self.relu3_3 = nn.ReLU()
        
        self.conv3_4 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_4 = nn.BatchNorm2d(128)
        self.relu3_4 = nn.ReLU()
        
        self.conv3_5 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_5 = nn.BatchNorm2d(128)
        self.relu3_5 = nn.ReLU()
        
        self.conv3_6 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_6 = nn.BatchNorm2d(128)
        self.relu3_6 = nn.ReLU()
        
        self.conv3_7 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_7 = nn.BatchNorm2d(128)
        self.relu3_7 = nn.ReLU()
        
        self.conv3_8 = nn.Conv2d(128, 128, kernel_size = 3, padding = 1)
        self.bn3_8 = nn.BatchNorm2d(128)
        self.relu3_8 = nn.ReLU()
        
        # 256 channels
        self.conv4_1 = nn.Conv2d(128, 256, kernel_size = 3, stride = 2, padding = 1)
        self.bn4_1 = nn.BatchNorm2d(256)
        self.relu4_1 = nn.ReLU()
        
        self.conv4_2 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_2 = nn.BatchNorm2d(256)
        self.relu4_2 = nn.ReLU()
        
        self.conv4_proj = nn.Conv2d(128, 256, kernel_size = 1) # for projection shortcut
        
        self.conv4_3 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_3 = nn.BatchNorm2d(256)
        self.relu4_3 = nn.ReLU()
        
        self.conv4_4 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_4 = nn.BatchNorm2d(256)
        self.relu4_4 = nn.ReLU()
        
        self.conv4_5 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_5 = nn.BatchNorm2d(256)
        self.relu4_5 = nn.ReLU()
        
        self.conv4_6 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_6 = nn.BatchNorm2d(256)
        self.relu4_6 = nn.ReLU()
        
        self.conv4_7 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_7 = nn.BatchNorm2d(256)
        self.relu4_7 = nn.ReLU()
        
        self.conv4_8 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_8 = nn.BatchNorm2d(256)
        self.relu4_8 = nn.ReLU()
        
        self.conv4_9 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_9 = nn.BatchNorm2d(256)
        self.relu4_9 = nn.ReLU()
        
        self.conv4_10 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_10 = nn.BatchNorm2d(256)
        self.relu4_10 = nn.ReLU()
        
        self.conv4_11 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_11 = nn.BatchNorm2d(256)
        self.relu4_11 = nn.ReLU()
        
        self.conv4_12 = nn.Conv2d(256, 256, kernel_size = 3, padding = 1)
        self.bn4_12 = nn.BatchNorm2d(256)
        self.relu4_12 = nn.ReLU()
        
        # 512 channels
        self.conv5_1 = nn.Conv2d(256, 512, kernel_size = 3, stride = 2)
        self.bn5_1 = nn.BatchNorm2d(512)
        self.relu5_1 = nn.ReLU()
        
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.bn5_2 = nn.BatchNorm2d(512)
        self.relu5_2 = nn.ReLU()
        
        self.conv5_proj = nn.Conv2d(256, 512, kernel_size = 1) # for projection shortcut
        
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.bn5_3 = nn.BatchNorm2d(512)
        self.relu5_3 = nn.ReLU()
        
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.bn5_4 = nn.BatchNorm2d(512)
        self.relu5_4 = nn.ReLU()
        
        self.conv5_5 = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.bn5_5 = nn.BatchNorm2d(512)
        self.relu5_5 = nn.ReLU()
        
        self.conv5_6 = nn.Conv2d(512, 512, kernel_size = 3, padding = 1)
        self.bn5_6 = nn.BatchNorm2d(512)
        self.relu5_6 = nn.ReLU()
        
        # global avg pooling
        self.Adaavgpool1 = nn.AdaptiveAvgPool2d((1, 1))
        
        # FC layer
        self.fc1 = nn.Linear(512, 1000)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.maxpool1(x)
        
        # 64 channels
        identity = x # input 기록
        
        x = self.conv2_1(x)
        x = self.bn2_1(x)
        x = self.relu2_1(x)
        
        x = self.conv2_2(x)
        x = self.bn2_2(x)
        x = self.relu2_2(x) + identity # 차원이 동일
        
        identity = x # input 기록
        
        x = self.conv2_3(x)
        x = self.bn2_3(x)
        x = self.relu2_3(x)
        
        x = self.conv2_4(x)
        x = self.bn2_4(x)
        x = self.relu2_4(x) + identity
        
        identity = x # input 기록
        
        x = self.conv2_5(x)
        x = self.bn2_5(x)
        x = self.relu2_5(x)
        
        x = self.conv2_6(x)
        x = self.bn2_6(x)
        x = self.relu2_6(x)
        
        identity = x # input 기록
        
        # 128 channels
        x = self.conv3_1(x)
        x = self.bn3_1(x)
        x = self.relu3_1(x)
        
        x = self.conv3_2(x)
        x = self.bn3_2(x)
        y = self.conv3_proj(identity) # 1x1 conv filter
        x = self.relu3_2(x + y)
        
        identity = x # input 기록
        
        x = self.conv3_3(x)
        x = self.bn3_3(x)
        x = self.relu3_3(x)
        
        x = self.conv3_4(x)
        x = self.bn3_4(x)
        x = self.relu3_4(x) + identity
        
        identity = x # input 기록
        
        x = self.conv3_5(x)
        x = self.bn3_5(x)
        x = self.relu3_5(x)
        
        x = self.conv3_6(x)
        x = self.bn3_6(x)
        x = self.relu3_6(x) + identity
        
        identity = x # input 기록
        
        x = self.conv3_7(x)
        x = self.bn3_7(x)
        x = self.relu3_7(x)
        
        x = self.conv3_8(x)
        x = self.bn3_8(x)
        x = self.relu3_8(x) + identity
        
        identity = x # input 기록
        
        # 256 channels
        x = self.conv4_1(x)
        x = self.bn4_1(x)
        x = self.relu4_1(x)
        
        x = self.conv4_2(x)
        x = self.bn4_2(x)
        y = self.conv4_proj(identity) # 1x1 filter 256개 사용
        x = self.relu4_2(x + y)
        
        identity = x
        
        x = self.conv4_3(x)
        x = self.bn4_3(x)
        x = self.relu4_3(x)
        
        x = self.conv4_4(x)
        x = self.bn4_4(x)
        x = self.relu4_4(x) + identity
        
        identity = x
        
        x = self.conv4_5(x)
        x = self.bn4_5(x)
        x = self.relu4_5(x)
        
        x = self.conv4_6(x)
        x = self.bn4_6(x)
        x = self.relu4_6(x) + identity
        
        identity = x
        
        x = self.conv4_7(x)
        x = self.bn4_7(x)
        x = self.relu4_7(x)
        
        x = self.conv4_8(x)
        x = self.bn4_8(x)
        x = self.relu4_8(x) + identity
        
        identity = x
        
        x = self.conv4_9(x)
        x = self.bn4_9(x)
        x = self.relu4_9(x)
        
        x = self.conv4_10(x)
        x = self.bn4_10(x)
        x = self.relu4_10(x) + identity
        
        identity = x
        
        x = self.conv4_11(x)
        x = self.bn4_11(x)
        x = self.relu4_11(x)
        
        x = self.conv4_12(x)
        x = self.bn4_12(x)
        x = self.relu4_12(x) + identity
        
        identity = x
        
        # 512 channels
        x = self.conv5_1(x)
        x = self.bn5_1(x)
        x = self.relu5_1(x)
        
        x = self.conv5_2(x)
        x = self.bn5_2(x)
        y = self.conv5_proj(identity)
        x = self.relu5_2(x + y)
        
        identity = x
        
        x = self.conv5_3(x)
        x = self.bn5_3(x)
        x = self.relu5_3(x)
        
        x = self.conv5_4(x)
        x = self.bn5_4(x)
        x = self.relu5_4(x) + identity
        
        identity = x
        
        x = self.conv5_5(x)
        x = self.bn5_5(x)
        x = self.relu5_5(x)
        
        x = self.conv5_6(x)
        x = self.bn5_6(x)
        x = self.relu5_6(x) + identity
        
        # global avg pool 2d
        x = self.Adaavgpool1(x)
        
        # Linear
        x = self.fc1(x)
        
        return x

Overwriting ResNet.py
